- Update: 2023.04.23

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# = = = = = = = = = = = = = = = = = =

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu-LS_Oil_Pan.xlsx')
list_product = df_menu['Product'].to_list()
print('总数量：' + str(len(list_product)))
print()

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['链接',
                           '车型',
                           'status'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for product in list_product:
    work.put_nowait(product)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        product = work.get_nowait()
        url = product
        driver = Chrome(service=Service(chromedriver), options=chrome_options)
        driver.maximize_window()
        time.sleep(0.3)
        driver.maximize_window()
        
        for i in range(31):
            try:
                vehicle = ''
                j = 0
                while True:
                    driver.get(url + '?page=' + str(j+1))
                    driver.maximize_window()
                    time.sleep(0.3)
                    WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="part__titre-groupe-vehicules"]')))
                    list_row = driver.find_elements(By.XPATH, '//*[@class="part__titre-groupe-vehicules"]')
                    for row in list_row:
                        vehicle += row.text + '\n'
                    
                    try:
                        driver.find_element(By.XPATH, '//*[@class="paginator__next"]')
                        j += 1
                    except:
                        break

                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'链接': [str(url)],
                                        '车型': [str(vehicle[:-1])],
                                        'status': ['ok']})
                
                # = = = = = = = = = = = = = = = = = =
                
                break

            except:
                if i == 30:
                    df_temp = pd.DataFrame({'链接': [str(url)],
                                            '车型': [''],
                                            'status': ['error']})
                continue

        # = = = = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(product, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        driver.close()
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./crawler-LS_Oil_Pan-vehicle.xlsx', index=False)
print()
print('搞定')

总数量：594

https://ecat.spectrapremium.com/en/parts/BMP02A  [0] - 剩余数量：589
https://ecat.spectrapremium.com/en/parts/BMP06A  [0] - 剩余数量：588
https://ecat.spectrapremium.com/en/parts/BMP08A  [0] - 剩余数量：587
https://ecat.spectrapremium.com/en/parts/BMP03A  [0] - 剩余数量：586
https://ecat.spectrapremium.com/en/parts/BMP09A  [0] - 剩余数量：585
https://ecat.spectrapremium.com/en/parts/BMP10A  [0] - 剩余数量：584
https://ecat.spectrapremium.com/en/parts/BMP07A  [0] - 剩余数量：583
https://ecat.spectrapremium.com/en/parts/BMP13A  [0] - 剩余数量：582
https://ecat.spectrapremium.com/en/parts/BMP12A  [0] - 剩余数量：581
https://ecat.spectrapremium.com/en/parts/BMP14A  [0] - 剩余数量：580
https://ecat.spectrapremium.com/en/parts/BMP05A  [0] - 剩余数量：579
https://ecat.spectrapremium.com/en/parts/BMP11A  [0] - 剩余数量：578
https://ecat.spectrapremium.com/en/parts/BMP16A  [0] - 剩余数量：577
https://ecat.spectrapremium.com/en/parts/BMP18A  [0] - 剩余数量：576
https://ecat.spectrapremium.com/en/parts/BMP19A  [0] - 剩余数量：575
https://ecat.spectrapremium.com

https://ecat.spectrapremium.com/en/parts/FP11A  [0] - 剩余数量：460
https://ecat.spectrapremium.com/en/parts/FP11B  [0] - 剩余数量：459
https://ecat.spectrapremium.com/en/parts/FP14B  [0] - 剩余数量：458
https://ecat.spectrapremium.com/en/parts/FP14A  [0] - 剩余数量：457
https://ecat.spectrapremium.com/en/parts/FP15B  [0] - 剩余数量：456
https://ecat.spectrapremium.com/en/parts/FP16B  [0] - 剩余数量：455
https://ecat.spectrapremium.com/en/parts/FP18A  [0] - 剩余数量：454
https://ecat.spectrapremium.com/en/parts/FP17A  [0] - 剩余数量：453
https://ecat.spectrapremium.com/en/parts/FP18B  [0] - 剩余数量：452
https://ecat.spectrapremium.com/en/parts/FP01B  [1] - 剩余数量：451
https://ecat.spectrapremium.com/en/parts/FP19B  [0] - 剩余数量：450
https://ecat.spectrapremium.com/en/parts/FP18D  [0] - 剩余数量：449
https://ecat.spectrapremium.com/en/parts/FP19C  [0] - 剩余数量：448
https://ecat.spectrapremium.com/en/parts/FP20A  [0] - 剩余数量：447
https://ecat.spectrapremium.com/en/parts/FP12A  [0] - 剩余数量：446
https://ecat.spectrapremium.com/en/parts/FP20B  [0] - 剩

https://ecat.spectrapremium.com/en/parts/GMP38A  [0] - 剩余数量：331
https://ecat.spectrapremium.com/en/parts/GMP40A  [0] - 剩余数量：330
https://ecat.spectrapremium.com/en/parts/GMP27A  [0] - 剩余数量：329
https://ecat.spectrapremium.com/en/parts/GMP46A  [0] - 剩余数量：328
https://ecat.spectrapremium.com/en/parts/GMP47A  [0] - 剩余数量：327
https://ecat.spectrapremium.com/en/parts/GMP29A  [0] - 剩余数量：326
https://ecat.spectrapremium.com/en/parts/GMP50A  [0] - 剩余数量：325
https://ecat.spectrapremium.com/en/parts/GMP51A  [0] - 剩余数量：324
https://ecat.spectrapremium.com/en/parts/GMP53A  [0] - 剩余数量：323
https://ecat.spectrapremium.com/en/parts/GMP52A  [0] - 剩余数量：322
https://ecat.spectrapremium.com/en/parts/GMP20B  [0] - 剩余数量：321
https://ecat.spectrapremium.com/en/parts/GMP55A  [0] - 剩余数量：320
https://ecat.spectrapremium.com/en/parts/GMP56A  [0] - 剩余数量：319
https://ecat.spectrapremium.com/en/parts/GMP57A  [0] - 剩余数量：318
https://ecat.spectrapremium.com/en/parts/GMP53B  [0] - 剩余数量：317
https://ecat.spectrapremium.com/en/parts

https://ecat.spectrapremium.com/en/parts/HYP30A  [0] - 剩余数量：202
https://ecat.spectrapremium.com/en/parts/HYP31A  [0] - 剩余数量：201
https://ecat.spectrapremium.com/en/parts/HYP17A  [0] - 剩余数量：200
https://ecat.spectrapremium.com/en/parts/HYP33A  [0] - 剩余数量：199
https://ecat.spectrapremium.com/en/parts/HYP32A  [0] - 剩余数量：198
https://ecat.spectrapremium.com/en/parts/HYP23A  [0] - 剩余数量：197
https://ecat.spectrapremium.com/en/parts/HYP34A  [0] - 剩余数量：196
https://ecat.spectrapremium.com/en/parts/KIP03A  [0] - 剩余数量：195
https://ecat.spectrapremium.com/en/parts/KIP02A  [0] - 剩余数量：194
https://ecat.spectrapremium.com/en/parts/MDP03A  [0] - 剩余数量：193
https://ecat.spectrapremium.com/en/parts/MDP04A  [0] - 剩余数量：192
https://ecat.spectrapremium.com/en/parts/MDP05A  [0] - 剩余数量：191
https://ecat.spectrapremium.com/en/parts/MDP01A  [0] - 剩余数量：190
https://ecat.spectrapremium.com/en/parts/MDP07A  [0] - 剩余数量：189
https://ecat.spectrapremium.com/en/parts/HYP27A  [0] - 剩余数量：188
https://ecat.spectrapremium.com/en/parts

https://ecat.spectrapremium.com/en/parts/TOP10B  [0] - 剩余数量：73
https://ecat.spectrapremium.com/en/parts/TOP22A  [0] - 剩余数量：72
https://ecat.spectrapremium.com/en/parts/TOP25A  [0] - 剩余数量：71
https://ecat.spectrapremium.com/en/parts/TOP28A  [0] - 剩余数量：70
https://ecat.spectrapremium.com/en/parts/TOP24A  [0] - 剩余数量：69
https://ecat.spectrapremium.com/en/parts/TOP28B  [0] - 剩余数量：68
https://ecat.spectrapremium.com/en/parts/TOP23A  [0] - 剩余数量：67
https://ecat.spectrapremium.com/en/parts/TOP29A  [0] - 剩余数量：66
https://ecat.spectrapremium.com/en/parts/TOP31A  [0] - 剩余数量：65
https://ecat.spectrapremium.com/en/parts/TOP32A  [0] - 剩余数量：64
https://ecat.spectrapremium.com/en/parts/TOP27A  [0] - 剩余数量：63
https://ecat.spectrapremium.com/en/parts/TOP33A  [0] - 剩余数量：62
https://ecat.spectrapremium.com/en/parts/TOP34B  [0] - 剩余数量：61
https://ecat.spectrapremium.com/en/parts/TOP26A  [0] - 剩余数量：60
https://ecat.spectrapremium.com/en/parts/TOP34A  [0] - 剩余数量：59
https://ecat.spectrapremium.com/en/parts/TOP35A  [0] - 

D:\Lennon\python\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
